# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile, os
import random
from glob import glob
import shutil
from PIL import Image

Mounted at /content/drive


In [ ]:
def unzip_to_folder(zip_path):
    folder_name = os.path.splitext(os.path.basename(zip_path))[0]
    os.makedirs(folder_name, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(folder_name)
    print(f"Unzipped {zip_path} → {folder_name}/")
    return folder_name

In [ ]:
unzip_to_folder("/content/drive/MyDrive/datasets/patches_sampled_50.zip")

extract_dir = "/content/patches_sampled_50"

Unzipped /content/drive/MyDrive/datasets/patches_sampled_50.zip → patches_sampled_50/


# Create training, validation, and test splits

In [ ]:
# Step 2. Create training, validation, and test splits
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory

# -----------------------------
# Parameters
# -----------------------------
data_dir = extract_dir
img_size = (256, 256)
batch_size = 64
seed = 42

# -----------------------------
# Create initial 80/20 split
# -----------------------------
train_ds = image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    validation_split=0.2,   # 20% for val+test
    subset='training',
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

val_test_ds = image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    validation_split=0.2,
    subset='validation',
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

# -----------------------------
# Split the 20% into 10% val + 10% test
# -----------------------------
val_batches = tf.data.experimental.cardinality(val_test_ds)
test_ds = val_test_ds.take(val_batches // 2)
val_ds = val_test_ds.skip(val_batches // 2)

print(f"Train batches: {tf.data.experimental.cardinality(train_ds)}")
print(f"Val batches:   {tf.data.experimental.cardinality(val_ds)}")
print(f"Test batches:  {tf.data.experimental.cardinality(test_ds)}")


Found 3000 files belonging to 3 classes.
Using 2400 files for training.
Found 3000 files belonging to 3 classes.
Using 600 files for validation.
Train batches: 38
Val batches:   5
Test batches:  5


In [ ]:
# Step 3. Prepare the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

# Normalization to rescales pixel intensity range (e.g., 0–255 → 0–1)
normalization_layer = tf.keras.layers.Rescaling(1./255)

def preprocess(ds):
    return ds.map(lambda x, y: (normalization_layer(x), y)).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds = preprocess(train_ds)
val_ds   = preprocess(val_ds)
test_ds  = preprocess(test_ds)

# Load the model and evaluate

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from file
model = load_model("/content/drive/MyDrive/liquefaction_cnn_v2.h5")

In [ ]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,365 (56.61 MB)

 Trainable params: 14,839,363 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.6666 - loss: 0.8363
Test accuracy: 0.6937500238418579


In [ ]:
from sklearn.metrics import classification_report

y_true, y_pred = [], []
for x, y in val_ds:
    preds = tf.argmax(model.predict(x), axis=1)
    y_true.extend(y.numpy())
    y_pred.extend(preds.numpy())

print(classification_report(y_true, y_pred, target_names=['low', 'moderate', 'high']))



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
              precision    recall  f1-score   support

         low       0.59      0.53      0.56        91
    moderate       0.62      0.81      0.70        83
        high       0.74      0.63      0.68       106

    accuracy                           0.65       280
   macro avg       0.65      0.66      0.65       280
weighted avg       0.66      0.65      0.65       280

